# Event Log Knowledge Graph (ELKG)

See `requirements.txt` for required dependencies.  
You can install the latest `eye` reasoner from https://github.com/eyereasoner/eye/tags.

In [1]:
import pm4py
import pandas as pd
import subprocess

from rdflib import Namespace

## CCEL (XES)

### Convert XES into ELKG

**(needs to be done only once!)**

In [ ]:
# - load XES

log = pm4py.read_xes("logs/xes/sepsis.xes")

In [ ]:
# - convert XES to RDF

from convert_log import convert_xes_rdf, RdfRepresent

convert_xes_rdf(log, "logs/xes/sepsis.ttl", Namespace("http://dutch.hospital.nl/sepsis#"), RdfRepresent.LINK_PRED)

### Test queries

In [ ]:
def run_query(query_file):
    cmd = f"eye n3/pqn.n3 --turtle logs/xes/sepsis.ttl --nope --query n3/queries/{query_file}"
    subprocess.run(cmd.split(" ")) 

In [ ]:
# - get traces based on activity occurrence, sequential relations, and arbitrary constraints

run_query("sepsis_query.n3")

## OCEL2

### Convert OCEL2 log into ELKG

In [ ]:
# - load OCEL2

log = pm4py.read_ocel2("logs/ocel2/ocel2-p2p.xml")

In [ ]:
# - convert OCEL2 to RDF

from convert_log import convert_ocel2_rdf

# (full log)
convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p.ttl", Namespace("http://ocel2.org/p2p#"))
# # (log subset with 5000 events)
# convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p-5000.ttl", Namespace("http://ocel2.org/p2p#"), 5000)
# # (log subset with 10000 events)
# convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p-10000.ttl", Namespace("http://ocel2.org/p2p#"), 10000)

#### Flatten based on perspective

In [ ]:
# - extract traces from ELKG using perspective (defined in traces_query.n3)

cmd = "eye n3/sort.n3 n3/traces/traces_collect.n3 --turtle logs/ocel2/ocel2-p2p.ttl --query n3/traces/traces_query.n3 --nope --skolem-genid trace"

with open("logs/ocel2/ocel2-p2p-traces.ttl", "w") as out_file: # file to store traces
    subprocess.run(cmd.split(" "), stdout=out_file) 

### Test queries

In [2]:
def run_query(query_file):
    cmd = f"eye n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/{query_file}"
    subprocess.run(cmd.split(" ")) 

In [ ]:
# - get all events and their activities in a trace
# (n3/queries/trace_activities_query.n3)

# use to check traces in query output.
# e.g., to lookup trace t_1, update trace ID in n3/queries/trace_activities_query.n3,
# and run code below

run_query("trace_activities_query.n3")

In [7]:
# - maverick buying (1)
# (see n3/queries/p2p_mav_buying1.n3)

run_query("p2p_mav_buying1.n3")

eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_mav_buying1.n3
EYE v10.17.3 (2024-08-07)
SWI-Prolog version 9.0.4
starting 25 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/pm/ELKG/n3/pqn.n3 SC=46
GET file:///Users/wvw/git/pm/ELKG/logs/ocel2/ocel2-p2p-expanded.ttl 

# Processed by EYE v10.17.3 (2024-08-07)
# eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_mav_buying1.n3

@prefix pq: <http://notation3.org/pqn#>.
@prefix skolem: <https://eyereasoner.github.io/.well-known/genid/trace#>.

pq:result pq:entry skolem:t_213.
pq:result pq:entry skolem:t_218.
pq:result pq:entry skolem:t_227.
pq:result pq:entry skolem:t_249.
pq:result pq:entry skolem:t_258.
pq:result pq:entry skolem:t_261.
pq:result pq:entry skolem:t_267.
pq:result pq:entry skolem:t_276.
pq:result pq:entry skolem:t_281.
pq:result pq:entry skolem:t_290.
pq:result pq:entry skolem:t_299.
pq:result pq:entry skolem:t_303.
pq:result pq:entry skolem:t_306.
pq:result pq:entry skolem:t_309.
pq:result pq:entry skolem:t_313.
pq:result pq:entry skolem:t_319.
pq:result pq:entry skolem:t_329.
pq:result pq:entry skolem:t_341.
pq:result pq:entry skolem:t_354.
pq:result pq:entry skolem:t_360.
pq:result pq:entry skolem:t_372.
pq:result pq:entry skolem:t_373.
pq:r

SC=333395
GET file:///Users/wvw/git/pm/ELKG/n3/queries/p2p_mav_buying1.n3 SC=1
networking 1266 [msec cputime] 1358 [msec walltime]
reasoning 66 [msec cputime] 69 [msec walltime]
2024-08-24T17:01:33.557Z in=333442 out=122 ent=122 step=122 brake=2 inf=12127601 sec=1.357 inf/sec=8937068



In [11]:
# - maverick buying (2)
# (see n3/queries/p2p_mav_buying2.n3)

# (nothing found!)
run_query("p2p_mav_buying2.n3")

eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_mav_buying2.n3
EYE v10.17.3 (2024-08-07)
SWI-Prolog version 9.0.4
starting 25 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/pm/ELKG/n3/pqn.n3 SC=46
GET file:///Users/wvw/git/pm/ELKG/logs/ocel2/ocel2-p2p-expanded.ttl 

# Processed by EYE v10.17.3 (2024-08-07)
# eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_mav_buying2.n3

# 2024-08-24T17:02:49.284Z in=333442 out=0 ent=0 step=0 brake=2 inf=11900706 sec=1.391 inf/sec=8555504
# ENDS



SC=333395
GET file:///Users/wvw/git/pm/ELKG/n3/queries/p2p_mav_buying2.n3 SC=1
networking 1311 [msec cputime] 1411 [msec walltime]
reasoning 55 [msec cputime] 57 [msec walltime]
2024-08-24T17:02:49.284Z in=333442 out=0 ent=0 step=0 brake=2 inf=11900706 sec=1.391 inf/sec=8555504



In [12]:
# - duplicate payments
# (see n3/queries/p2p_duplic_paym.n3)

run_query("p2p_duplic_paym.n3")

eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_duplic_paym.n3
EYE v10.17.3 (2024-08-07)
SWI-Prolog version 9.0.4
starting 25 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/pm/ELKG/n3/pqn.n3 SC=46
GET file:///Users/wvw/git/pm/ELKG/logs/ocel2/ocel2-p2p-expanded.ttl 

# Processed by EYE v10.17.3 (2024-08-07)
# eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_duplic_paym.n3

@prefix pq: <http://notation3.org/pqn#>.
@prefix skolem: <https://eyereasoner.github.io/.well-known/genid/trace#>.

pq:result pq:entry (skolem:t_3 "# payments: 2").
pq:result pq:entry (skolem:t_13 "# payments: 2").
pq:result pq:entry (skolem:t_23 "# payments: 2").
pq:result pq:entry (skolem:t_26 "# payments: 2").
pq:result pq:entry (skolem:t_29 "# payments: 2").
pq:result pq:entry (skolem:t_32 "# payments: 2").
pq:result pq:entry (skolem:t_46 "# payments: 2").
pq:result pq:entry (skolem:t_51 "# payments: 4").
pq:result pq:entry (skolem:t_56 "# payments: 2").
pq:result pq:entry (skolem:t_65 "# payments: 3").
pq:result pq:entry (skolem:t_85 "# payments: 2").
pq:result pq:entry (skolem:t_99 "# payments: 2").
pq:result pq:entry (skolem:t_108 "# payments: 2").
pq:result pq:entry (skolem:t_119 "# payments: 2").
pq:result pq:entry (skolem:t_

SC=333395
GET file:///Users/wvw/git/pm/ELKG/n3/queries/p2p_duplic_paym.n3 SC=1
networking 1300 [msec cputime] 1393 [msec walltime]
reasoning 36 [msec cputime] 37 [msec walltime]
2024-08-24T17:03:31.674Z in=333442 out=188 ent=188 step=188 brake=2 inf=11960509 sec=1.361 inf/sec=8788030



In [19]:
# - lengthy approval process
# (from creation to approval of purchase order)
# (see n3/queries/p2p_long_approv.n3)

run_query("p2p_long_approv.n3")

eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_long_approv.n3
EYE v10.17.3 (2024-08-07)
SWI-Prolog version 9.0.4
starting 25 [msec cputime] 28 [msec walltime]
GET file:///Users/wvw/git/pm/ELKG/n3/pqn.n3 SC=47
GET file:///Users/wvw/git/pm/ELKG/logs/ocel2/ocel2-p2p-expanded.ttl 

# Processed by EYE v10.17.3 (2024-08-07)
# eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_long_approv.n3

@prefix pq: <http://notation3.org/pqn#>.
@prefix skolem: <https://eyereasoner.github.io/.well-known/genid/trace#>.

pq:result pq:entry (skolem:t_0 "19 days").
pq:result pq:entry (skolem:t_3 "16 days").
pq:result pq:entry (skolem:t_63 "15 days").
pq:result pq:entry (skolem:t_141 "15 days").
pq:result pq:entry (skolem:t_247 "18 days").
pq:result pq:entry (skolem:t_505 "18 days").
pq:result pq:entry (skolem:t_563 "17 days").
pq:result pq:entry (skolem:t_689 "16 days").
pq:result pq:entry (skolem:t_695 "16 days").
pq:result pq:entry (skolem:t_833 "21 days").
pq:result pq:entry (skolem:t_855 "16 days").
pq:result pq:entry (skolem:t_856 "25 days").
pq:result pq:entry (skolem:t_915 "24 days").
# 2024-08-24T17:12:44.158Z in=333443 out=13 ent=13 step=13 brake=1 inf=13387761 sec=1.462 inf/sec=9157155
# ENDS



SC=333395
GET file:///Users/wvw/git/pm/ELKG/n3/queries/p2p_long_approv.n3 SC=1
networking 1300 [msec cputime] 1417 [msec walltime]
reasoning 137 [msec cputime] 142 [msec walltime]
2024-08-24T17:12:44.158Z in=333443 out=13 ent=13 step=13 brake=1 inf=13387761 sec=1.462 inf/sec=9157155



In [20]:
# - lengthy process overall 
# (from initial purchase requisition to final payment)
# (see n3/queries/p2p_long_process.n3)

run_query("p2p_long_process.n3")

eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_long_process.n3
EYE v10.17.3 (2024-08-07)
SWI-Prolog version 9.0.4
starting 24 [msec cputime] 26 [msec walltime]
GET file:///Users/wvw/git/pm/ELKG/n3/pqn.n3 SC=47
GET file:///Users/wvw/git/pm/ELKG/logs/ocel2/ocel2-p2p-expanded.ttl 

# Processed by EYE v10.17.3 (2024-08-07)
# eye --quiet n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/p2p_long_process.n3

@prefix pq: <http://notation3.org/pqn#>.
@prefix skolem: <https://eyereasoner.github.io/.well-known/genid/trace#>.

pq:result pq:entry (skolem:t_266 "44 days").
pq:result pq:entry (skolem:t_288 "43 days").
pq:result pq:entry (skolem:t_343 "44 days").
pq:result pq:entry (skolem:t_424 "51 days").
pq:result pq:entry (skolem:t_443 "45 days").
pq:result pq:entry (skolem:t_443 "46 days").
pq:result pq:entry (skolem:t_485 "49 days").
pq:result pq:entry (skolem:t_485 "52 days").
pq:result pq:entry (skolem:t_528 "43 days").
pq:result pq:entry (skolem:t_563 "46 days").
pq:result pq:entry (skolem:t_660 "46 days").
pq:result pq:entry (skolem:t_687 "66 days").
pq:result pq:entry (skolem:t_689 "54 days").
pq:result pq:entry (skolem:t_759 "47 days").
pq:result pq:entry (skolem:t_767 "43 days").
pq:result pq:entry (skolem:t_793 "49 days").
pq:result

SC=333395
GET file:///Users/wvw/git/pm/ELKG/n3/queries/p2p_long_process.n3 SC=1
networking 1304 [msec cputime] 1401 [msec walltime]
reasoning 42 [msec cputime] 42 [msec walltime]
2024-08-24T17:14:08.813Z in=333443 out=23 ent=23 step=24 brake=1 inf=12279501 sec=1.370 inf/sec=8963139

